# Dynamics lineshapes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.kinematics.phasespace import Kallen
from ampform_dpd import DynamicsBuilder, create_mass_symbol_mapping
from ampform_dpd.decay import ThreeBodyDecay
from ampform_dpd.dynamics import (
    BlattWeisskopf,
    BreitWignerMinL,
    BuggBreitWigner,
    EnergyDependentWidth,
    FlattéSWave,
    P,
    Q,
)
from ampform_dpd.dynamics.builder import get_mandelstam_s
from ampform_dpd.io import cached
from matplotlib_inline.backend_inline import set_matplotlib_formats

from polarimetry.io import (
    display_doit,
    display_latex,
    mute_ampform_warnings,
    mute_jax_warnings,
)
from polarimetry.lhcb import load_model_parameters, load_three_body_decay
from polarimetry.lhcb.dynamics import (
    formulate_breit_wigner,
    formulate_bugg_breit_wigner,
    formulate_exponential_bugg_breit_wigner,
    formulate_flatte_1405,
)
from polarimetry.lhcb.particle import load_particles
from polarimetry.lhcb.symbol import create_meson_radius_symbol
from polarimetry.plot import use_mpl_latex_fonts


def load_parameters(
    decay: ThreeBodyDecay, model_id: int | str = 0
) -> dict[sp.Symbol, complex | float]:
    return load_model_parameters(
        decay=decay,
        filename="../../data/model-definitions.yaml",
        model_id=model_id,
        particle_definitions=particles,
    )


particles = load_particles("../../data/particle-definitions.yaml")
ls_model_id = "Alternative amplitude model obtained using LS couplings"
Sigma = particles["Sigma-"]
K = particles["K-"]
Lambda_c = particles["Lambda_c+"]
p = particles["p"]
pi = particles["pi+"]
mute_jax_warnings()
mute_ampform_warnings()
set_matplotlib_formats("svg")
use_mpl_latex_fonts()

In the following, we consider a decay $0 \xrightarrow{L} (r \xrightarrow{\ell} ij)k$, where the resonance&nbsp;$r$ is produced at the **production vertex** with angular momentum $L$ and decays at the **decay vertex** with angular momentum $\ell$. The meson radii for both vertices are denoted $R_\mathrm{prod}$ and $R_\mathrm{dec}$, respectively.

## Relativistic Breit–Wigner

Most resonances were parametrized with a standard relativistic Breit–Wigner with two form factors for the vertices.

In [ ]:
s, mr, Γr, mi, mj = sp.symbols("s m_r Gamma_R m_i m_j", nonnegative=True)
m0, mk = sp.symbols("m0 m_k")
R_prod = create_meson_radius_symbol("prod")
R_dec = create_meson_radius_symbol("dec")
L, ell = sp.symbols("L ell", integer=True, nonnegative=True)
display_doit(BreitWignerMinL(s, m0, mk, mr, Γr, mi, mj, ell, L, R_dec, R_prod))

## Bugg Breit–Wigner

The $K(700)$ and $K(1430)$ were modeled with a Bugg Breit–Wigner function. Here, $s_A$ is the **Adler zero**.

In [ ]:
gamma, sA = sp.symbols("gamma s_A")
bugg = BuggBreitWigner(s, mr, Γr, mi, mj, gamma)
sA_expr = mi**2 - mj**2 / 2
display_latex({
    bugg: bugg.evaluate().subs(sA_expr, sA),
    sA: sA_expr,
})

One of the models uses a Bugg Breit–Wigner with an exponential factor (see [this paper, Eq. (4)](https://arxiv.org/pdf/hep-ex/0510019.pdf#page=3)):

In [ ]:
q = Q(s, m0, mk)
alpha = sp.Symbol("alpha")
bugg * sp.exp(-alpha * q**2)

## Flatté for S-waves

In the original amplitude analysis, the $\varLambda(1405)$&nbsp;baryon was modeled with a Flatté for $S$-waves ($\ell=0$), coupled to the channel $\varLambda(1405) \to \varSigma^-\pi^+$. Note that both the $\varSigma^-\pi^+$ threshold and the mass of the $\varLambda(1405)$ lies below the $pK^-$ threshold.

In [ ]:
Γ1, Γ2, mπ, mΣ = sp.symbols("Gamma1 Gamma2 m_pi m_Sigma")
display_doit(FlattéSWave(s, mr, (Γ1, Γ2), (mi, mj), (mπ, mΣ)))

## Other definitions

In [ ]:
x, y, z = sp.symbols("x:z")
exprs = [
    EnergyDependentWidth(s, mr, Γr, mi, mj, ell, R_dec),
    BlattWeisskopf(z, ell),
    P(s, mi, mj),
    Q(s, m0, mk),
    Kallen(x, y, z),
]
display_latex({x: x.doit(deep=False) for x in exprs})

## Lineshape visualizations

The lineshapes are visualized below for some of the relevant resonances. The section of lineshapes for each resonance can be found [`model-definitions.yaml`](../../data/model-definitions.yaml)

In [ ]:
def plot_lineshape(
    dynamics_builder: DynamicsBuilder,
    title: str,
    resonances: str,
    x_range: tuple[float, float],
    y_title: float,
    legend_position: tuple[int, tuple[float, float]],
    model_id: int | str = 0,
) -> list[list[complex]]:
    min_ls = len(resonances) != 1
    if not min_ls:
        title = f"{title} for {resonances[0]}"
    decay = load_three_body_decay(resonances, particles, min_ls)
    parameter_defaults = load_parameters(decay, model_id)
    parameter_defaults.update(create_mass_symbol_mapping(decay))
    display_latex({c: dynamics_builder(c)[0].doit(deep=False) for c in decay.chains})
    fig, axes = plt.subplots(
        figsize=(6, 2 * len(decay.chains)),
        nrows=len(decay.chains),
        sharex=True,
    )
    fig.patch.set_facecolor("none")
    for ax in axes:
        ax.patch.set_facecolor("none")
        ax.spines["bottom"].set_position("zero")
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
    fig.suptitle(title, y=y_title)
    x = np.linspace(*x_range, num=300)
    test_arrays = []
    for ax, chain in zip(axes, decay.chains, strict=True):
        expr, parameters = dynamics_builder(chain)
        func = cached.lambdify(
            expr.doit(),
            parameters=parameters | parameter_defaults,
            backend="jax",
        )
        mass_name = f"m_{{{chain.resonance.latex}}}"
        resonance_mass = func.parameters[mass_name]
        child1, child2 = chain.decay_products
        phsp_min = child1.mass + child2.mass
        phsp_max = chain.parent.mass - chain.spectator.mass
        ax.axvspan(x.min(), phsp_min, alpha=0.15, color="gray", label="non-physical")
        ax.axvspan(phsp_max, x.max(), alpha=0.15, color="gray")
        ax.axvline(resonance_mass, c="red", ls="dotted", label=R"$m_\mathrm{res}$")
        sigma = get_mandelstam_s(chain.decay_node)
        z = func({sigma.name: x**2})
        ax.plot(x, np.abs(z), label="abs")
        ax.plot(x, z.imag, label="imag")
        ax.plot(x, z.real, label="real")
        if min_ls:
            axis_title = f"${chain.resonance.latex}$"
        else:
            axis_title = Rf"$L={chain.incoming_ls.L}, \ell={chain.outgoing_ls.L}$"
        ax.set_title(axis_title, y=0.86)
        ax.set_xlim(*x_range)
        test_arrays.append(z[80:-40:50].tolist())
    legend_axis, anchor = legend_position
    axes[legend_axis].legend(
        bbox_to_anchor=anchor,
        framealpha=1,
        handlelength=1,
        handletextpad=0.5,
        loc="upper right",
    )
    fig.tight_layout()
    plt.show()
    return test_arrays

### Breit–Wigner with form factors

In [ ]:
arrays = plot_lineshape(
    formulate_breit_wigner,
    title="Relativistic Breit–Wigner with form factors",
    resonances=["K(892)"],
    x_range=(0.5, 1.5),
    y_title=0.97,
    model_id=ls_model_id,
    legend_position=(0, (1.03, 1.03)),
)
expected = [
    [
        (3.2421271800994873 + 0.2397163361310959j),
        (-9.898703575134277 + 7.095293998718262j),
        (-3.115615129470825 + 0.6948883533477783j),
        (-1.87081778049469 + 0.3027965724468231j),
    ],
    [
        (3.266894817352295 + 0.241547629237175j),
        (-9.865890502929688 + 7.071774482727051j),
        (-3.026120185852051 + 0.6749279499053955j),
        (-1.613049864768982 + 0.2610761523246765j),
    ],
    [
        (3.266894817352295 + 0.241547629237175j),
        (-9.865890502929688 + 7.071774482727051j),
        (-3.026120185852051 + 0.6749279499053955j),
        (-1.613049864768982 + 0.2610761523246765j),
    ],
    [
        (3.3342278003692627 + 0.2465260773897171j),
        (-9.776321411132812 + 7.007572650909424j),
        (-2.783756971359253 + 0.6208727359771729j),
        (-1.0391007661819458 + 0.16818106174468994j),
    ],
]
np.testing.assert_equal(arrays, expected, err_msg=f"Full array: {arrays}")

### Bugg Breit–Wigner

In [ ]:
arrays = plot_lineshape(
    formulate_bugg_breit_wigner,
    title="Bugg Breit–Wigner",
    resonances=["K(700)", "K(1430)"],
    x_range=(0.5, 1.5),
    y_title=0.92,
    legend_position=(0, (1.03, 1.15)),
)
expected = [
    [
        (2.208456516265869 + 4.439322471618652j),
        (-1.9505517482757568 + 2.491237163543701j),
        (-1.473872423171997 + 0.7617897987365723j),
        (-0.9912535548210144 + 0.28465157747268677j),
    ],
    [
        (0.7669709324836731 + 0.03260128200054169j),
        (0.9742231369018555 + 0.09491971880197525j),
        (1.4088435173034668 + 0.3140260875225067j),
        (2.2870097160339355 + 1.7166467905044556j),
    ],
]
np.testing.assert_equal(arrays, expected, err_msg=f"Full array: {arrays}")

In [ ]:
arrays = plot_lineshape(
    formulate_exponential_bugg_breit_wigner,
    title="Exponential Bugg Breit–Wigner",
    resonances=["K(700)", "K(1430)"],
    x_range=(0.5, 1.5),
    y_title=0.92,
    legend_position=(0, (1.03, 1.15)),
    model_id="Alternative amplitude model with an additional overall exponential form factor exp(-alpha q^2) multiplying Bugg lineshapes. The exponential parameter is indicated as alpha",
)
expected = [
    [
        (2.546529531478882 + 5.48952054977417j),
        (-2.065908193588257 + 2.9268152713775635j),
        (-1.5488947629928589 + 0.9246438145637512j),
        (-0.9949549436569214 + 0.3458932340145111j),
    ],
    [
        (0.6374956965446472 + 0.029078781604766846j),
        (0.8470211625099182 + 0.09163091331720352j),
        (1.2738444805145264 + 0.3283860683441162j),
        (1.901564121246338 + 1.7310668230056763j),
    ],
]
np.testing.assert_equal(arrays, expected, err_msg=f"Full array: {arrays}")

### Flatté for $\varLambda(1405)$

In [ ]:
arrays = plot_lineshape(
    formulate_flatte_1405,
    title="S-wave Flatté",
    resonances=["L(1405)"],
    x_range=(1.3, 2.3),
    y_title=0.94,
    legend_position=(1, (1.08, 1.02)),
    model_id=ls_model_id,
)
expected = [
    [
        (-1.6116788387298584 + 0.8600924015045166j),
        (-0.8681501150131226 + 0.29150354862213135j),
        (-0.5734393000602722 + 0.14183905720710754j),
        (-0.41758209466934204 + 0.08123786002397537j),
    ],
    [
        (-1.5881913900375366 + 0.847558319568634j),
        (-0.8303629755973816 + 0.2788156569004059j),
        (-0.5090504884719849 + 0.12591268122196198j),
        (-0.2696514427661896 + 0.052458930760622025j),
    ],
]
np.testing.assert_equal(arrays, expected, err_msg=f"Full array: {arrays}")